<a href="https://colab.research.google.com/github/englissi/englissi/blob/main/%EB%9F%AC%EC%8B%9C%EC%95%84%EC%96%B4%EB%B0%9C%EC%9D%8C%ED%8F%89%EA%B0%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install gradio speechrecognition
!pip install gradio gtts

In [ ]:
import gradio as gr
from gtts import gTTS
import speech_recognition as sr
from difflib import SequenceMatcher
import tempfile
import os

def tts_russian(word):
    tts = gTTS(text=word, lang='ru')
    temp_file_path = tempfile.mktemp(suffix=".mp3")
    tts.save(temp_file_path)
    return temp_file_path

def recognize_speech_from_microphone_russian(audio_path):
    recognizer = sr.Recognizer()
    try:
        with sr.AudioFile(audio_path) as source:
            audio_data = recognizer.record(source)
            text = recognizer.recognize_google(audio_data, language='ru-RU')
            return text
    except sr.UnknownValueError:
        return "Could not understand the audio"
    except sr.RequestError as e:
        return f"Could not request results from Google Speech Recognition service; {e}"
    except Exception as e:
        return str(e)

def calculate_similarity_russian(word, recognized_text):
    return SequenceMatcher(None, word, recognized_text).ratio() * 100

def process_audio_russian(word, audio_path):
    recognized_text = recognize_speech_from_microphone_russian(audio_path)
    if "Error" in recognized_text or "Could not" in recognized_text:
        return recognized_text, 0.0
    similarity = calculate_similarity_russian(word, recognized_text)
    return recognized_text, similarity

def evaluate_pronunciation_russian(word):
    temp_file_path = tts_russian(word)
    return temp_file_path

def process_all_russian(word, audio_path):
    recognized_text, similarity = process_audio_russian(word, audio_path)
    return recognized_text, similarity

with gr.Blocks() as demo:
    with gr.Row():
        word_input = gr.Textbox(label="Введите слово для оценки произношения")
        tts_button = gr.Button("Прослушайте слово")
    tts_audio = gr.Audio(label="Оригинальное произношение", type="filepath")

    with gr.Row():
        mic_input = gr.Audio(label="Ваше произношение", type="filepath")
        result_button = gr.Button("Оценить произношение")

    recognized_text = gr.Textbox(label="Распознанный текст")
    similarity_score = gr.Number(label="Сходство (%)")

    tts_button.click(evaluate_pronunciation_russian, inputs=word_input, outputs=tts_audio)
    result_button.click(process_all_russian, inputs=[word_input, mic_input], outputs=[recognized_text, similarity_score])

demo.launch()
